In [74]:
import os
import keras 
import numpy as np
from keras import layers
from keras import models 
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img

In [75]:
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    return model

In [76]:
model=create_model()

In [77]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 12, 12, 64)      

In [78]:
# Generating images for the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                  )

# Generating images for the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)

# Creating the Training set
training_set = train_datagen.flow_from_directory(r'C:\Users\user\dataset\train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

# Creating the Test set
test_set = test_datagen.flow_from_directory(r'C:\Users\user\dataset\test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')              

Found 1975 images belonging to 3 classes.
Found 450 images belonging to 3 classes.


In [79]:
training_set.class_indices

{'healthy_wheat': 0, 'leaf_rust': 1, 'stem_rust': 2}

In [80]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#to save model
checkpoint_path = r"C:\Users\user\dataset\training_1\cp3.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_best_only=True)


In [ ]:
# Training the CNN on the Training set and evaluating it on the Test set
model.fit(training_set,
                  steps_per_epoch = 128,
                  epochs = 10,
                  validation_data = test_set,
                  validation_steps = 64,
                  callbacks=[cp_callback]
                  )

Epoch 1/10
 51/128 [==========>...................] - ETA: 10:32:01 - loss: 0.7579 - accuracy: 0.6248